### Sources 

https://baseballsavant.mlb.com/leaderboard

https://www.retrosheet.org/downloads/othercsvs.html

https://github.com/chadwickbureau/register/tree/master

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import glob 

In [ ]:
# Read files 
pitchers = pd.read_csv("datasets/pitchers.csv")
batters = pd.read_csv("datasets/batters.csv")
plays = pd.concat(map(pd.read_csv, glob.glob("datasets/retrosheet/*plays.csv")), ignore_index=True) 
register = pd.concat(map(pd.read_csv, glob.glob("datasets/chadwick_bureau_register/people-*.csv")), ignore_index=True)

In [ ]:
# Extract ids for matching 
mlb_ids = np.unique(np.concatenate((pitchers["player_id"], batters["player_id"])))
print(f"Number of unique MlB ids: {len(mlb_ids)}")

retro_ids = np.unique(np.concatenate((plays["batter"], plays["pitcher"])).astype(str))
print(f"Number of unique retrosheet ids: {len(retro_ids)}")

player_id_map = register[(register['key_mlbam'].isin(mlb_ids) | register['key_retro'].isin(retro_ids))]

Number of unique MlB ids: 3651
Number of unique retrosheet ids: 3776
